# Import Library 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,Flatten,Dense
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from PIL import Image
import sys

## Import data from directory 

In [ ]:
import os
image_id = []
object_type = []
dir = '/content/drive/MyDrive/data/'
for dirname in os.listdir(dir):
    if dirname!='.DS_Store':
        for filename in os.listdir(dir+dirname):
              image_id.append(dir+dirname+'/'+filename)
              object_type.append(dirname)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(image_id, object_type)),columns = ['ImageID','Type'])

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
inp = df['ImageID']
out = df['Type']

In [ ]:
train_in,test_in,train_out,test_out = train_test_split(inp,out,random_state = 42,test_size = 0.2,stratify = out)      #splitting the data set   

# Create an object from image data generator

In [ ]:
data_gen = ImageDataGenerator(
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest'
)

# Catenation of training input and output

In [ ]:
train_x = pd.concat([train_in,train_out],axis=1)

# Datasets value 

In [ ]:
from collections import Counter 
Counter(object_type)

Counter({'cat': 1185, 'kenye west': 1245, 'pikachu': 1040})

# Train images from whole dataset remain after spiliting 

In [ ]:
train_generator = data_gen.flow_from_dataframe(
                    train_x ,x_col = "ImageID",y_col = "Type",
                    target_size = (200,200),
                    batch_size = 64)

Found 2776 validated image filenames belonging to 3 classes.


# Cancatinate the test input and test output 

In [ ]:
test_x = pd.concat([test_in,test_out],axis=1)

# No. of image for  validation after splittting

In [ ]:
val_generator = data_gen.flow_from_dataframe(
                    test_x,x_col = "ImageID",y_col = "Type",
                    target_size = (200,200),
                    batch_size = 64)

Found 694 validated image filenames belonging to 3 classes.


## Model - Using ResNET50 as transfer learining model

In [ ]:
model = models.Sequential()
model.add(ResNet50(input_shape = (200,200, 3), include_top = False, weights = 'imagenet'))
model.add(layers.MaxPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(3, activation = 'softmax'))

94773248/94765736 [==============================] - 1s 0us/step


# Complile of model with categorical crossentropy and adam optimizer with learning rate 0.001

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(learning_rate = 0.001),
             metrics = ['accuracy'])

# Fitting the model with epoch 10 and batch size 64

In [ ]:
history = model.fit(train_generator,epochs = 10,
                    validation_data = val_generator)

Epoch 1/10
44/44 [==============================] - 617s 14s/step - loss: 11.2146 - accuracy: 0.7313 - val_loss: 49.1269 - val_accuracy: 0.5072
Epoch 2/10
44/44 [==============================] - 46s 1s/step - loss: 0.5351 - accuracy: 0.8894 - val_loss: 120.7422 - val_accuracy: 0.4914
Epoch 3/10
44/44 [==============================] - 46s 1s/step - loss: 0.2139 - accuracy: 0.9282 - val_loss: 27.6346 - val_accuracy: 0.7435
Epoch 4/10
44/44 [==============================] - 46s 1s/step - loss: 0.1881 - accuracy: 0.9362 - val_loss: 0.6941 - val_accuracy: 0.8790
Epoch 5/10
44/44 [==============================] - 45s 1s/step - loss: 0.1028 - accuracy: 0.9648 - val_loss: 0.0999 - val_accuracy: 0.9697
Epoch 6/10
44/44 [==============================] - 45s 1s/step - loss: 0.0758 - accuracy: 0.9734 - val_loss: 0.1150 - val_accuracy: 0.9582
Epoch 7/10
44/44 [==============================] - 46s 1s/step - loss: 0.0767 - accuracy: 0.9747 - val_loss: 0.0805 - val_accuracy: 0.9769
Epoch 8/10
44

## Prediction on Validation 

In [ ]:
pred= model.predict_classes(val_generator)
pred

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2, 0, 1,
       0, 1, 2, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 2, 1, 2,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 1, 1,
       2, 0, 0, 1, 0, 1, 2, 1, 0, 0, 2, 1, 2, 2, 1, 1, 2, 0, 0, 1, 1, 2,
       1, 2, 0, 2, 2, 2, 1, 2, 1, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 1,
       0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 0, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 0, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 1,
       0, 0, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 0, 2, 2, 1, 0,
       1, 2, 2, 2, 1, 1, 2, 1, 0, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 0, 2, 1,
       1, 1, 2, 2, 0, 2, 2, 1, 0, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 2, 1, 0,
       1, 1, 0, 1, 2, 0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 2, 1, 0, 0, 1, 1, 0, 2, 1, 1, 1, 2, 2, 2,

# Mapping of array with their lables 

In [ ]:
pred_1= pd.Series(pred).map({0:'Cat', 1:'kenye west',2:'pikachu'})


# Predict the image with class

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict(path):
  img = image.load_img(path=path ,target_size=(200, 200,3))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  pred = model.predict(img_batch)
  prediction= np.argmax(pred,axis=1)
  class_indices = list(val_generator.class_indices.keys())
  return (class_indices[prediction[0]],max(pred[0])*100)

# Prediction with Probability

In [ ]:
predict('/content/drive/MyDrive/data/kenye west/1000.jpg')

('kenye west', 99.9506950378418)

In [ ]:
from sklearn.metrics import confusion_matrix
output = []
for x in test_in:
  output.append(predict(x)[0])

# Print 10 predicted output

In [ ]:
output[:10]

['pikachu',
 'kenye west',
 'kenye west',
 'cat',
 'kenye west',
 'cat',
 'kenye west',
 'kenye west',
 'cat',
 'cat']